In [1]:
import numpy as np
import pandas as pd 
import json

import MeCab

In [2]:
m = MeCab.Tagger(' -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')

In [3]:
with open("../data/coco_reviews.json", "r") as f:
    json_dict = json.load(f)

In [4]:
def wakachi(text):
    splitted = ' '.join([
        x.split('\t')[0] for x in m.parse(text).splitlines()[:-1] 
            if x.split('\t')[1].split(',')[0] not in ['助詞', '助動詞', '接続詞', '動詞', '記号']
    ])
    return splitted

In [5]:
def set_stopwords():
    with open ("nlp/stopword_japanese.txt", "r") as f:
        stopwords = [line.strip() for line in f]
        stopwords = [ss for ss in stopwords if not ss==u'']

    return stopwords
# set_stopwords()
stopwords = set_stopwords()

In [6]:
def join_list_str(list):
    return ' '.join(list)

def exclude_stopword(text):
    changed_text = [token for token in text.lower().split(" ") if token != "" if token not in stopwords]
    changed_text = join_list_str(changed_text)
    return changed_text

In [66]:
reviews = []
for i in range(1, len(json_dict)+1):
    # print(json_dict[str(i)]["title"])
    # print(json_dict[str(i)]["reviews"])
    reviews_info = json_dict[str(i)]["reviews"]
    all_review_str = join_list_str([exclude_stopword(wakachi(review_info["review"])) for review_info in reviews_info])
    reviews += [all_review_str]
    # print(i)

In [12]:
from sklearn import feature_extraction

In [94]:
# max_dfとmin_dfによって結果は大きく変わる
vectorizer = feature_extraction.text.TfidfVectorizer(
    decode_error="ignore",
    strip_accents="unicode",
    lowercase=True,
    analyzer="word",
    max_df=0.8,
    min_df=0.3
)

In [95]:
X = vectorizer.fit_transform(reviews).toarray()
index = X.argsort(axis=1)[:, ::-1]
feature_names = np.array(vectorizer.get_feature_names())
feature_words = feature_names[index]

In [97]:
import csv
with open("nlp/tfidf.csv", "w") as f:
    writer = csv.writer(f, lineterminator=",")
    writer.writerows(feature_words)